# PyPSA-Eur style power system plot

## Authored by F.Hofmann and D.Fioriti

The notebook reproduces the plot like a beautiful scheme of the
European Transmission System published in https://arxiv.org/abs/1806.01613.

Original version by F.Hofmann, revised by D.Fioriti

In [ ]:
#"""
#Created on Mon Sep 19 15:51:31 2022
#
#@author: fabian and davide-f
#"""

In [ ]:
# change current directory to parent folder
import os
import sys

if not os.path.isdir("pypsa-earth"):
    os.chdir("../../..")
sys.path.append(os.getcwd()+"/pypsa-earth/scripts")

In [ ]:
import pypsa
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from datetime import datetime
from cartopy import crs as ccrs
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
import math

Two files are needed:
* PyPSA network file (e.g. "elec.nc" contains a lot of details and looks perfect)
* a country shape file (may by found in "resources/shapes/country_shapes.geojson")

### Input parameters

In [ ]:
scenario_name = "ID"  # scenario name, default value is "" for tutorial or default configuration
                    # value shall be non null if a scenario name is specified under the "run" tag in the config file

path_imgs = "./documentation/notebooks/viz"

figsize = (8,8)
max_node_size = 0.6
ref_area_node = 3700675068162.292969  # reference size: ES
max_line_size = 5

### Define utility functions

In [ ]:
def get_scenario_network_and_regions(scenario_name):
    """Load the network and onshore region for a scenario_name"""
    scenario_subpath = scenario_name + "/" if scenario_name else ""
    n = pypsa.Network("./pypsa-earth/networks/" + scenario_subpath + "elec.nc")
    regions_onshore = gpd.read_file("./pypsa-earth/resources/" + scenario_subpath + "shapes/country_shapes.geojson")

    return n, regions_onshore

def label_objs_legend(values):
    """Create labels with units in GW or MW depending on the numeric value of values"""
    if min(values) > 1000.:
        return ["{:d} GW".format(int(v/1000)) for v in values]
    elif max(values) >= 250.:
        return ["{:.1f} GW".format(v/1000) for v in values]
    else:
        return ["{:d} MW".format(round(v)) for v in values]

def normalize_obj(values, max_value_output, max_value_input=1):
    """Normalize an array or pandas series 'values' to have maximum value 'max_value'"""
    if isinstance(values, pd.Series) and not values.empty:
        return values / max_value_input * max_value_output
    elif isinstance(values, list) and not values:
        return values
    else:
        return [v / max_value_input * max_value_output for v in values]

def get_nice_extent(bounds, factor=1.):
    """Get extent that is nice to see. It ensures the desired size factor width/height"""
    desired_extent = bounds[[0, 2, 1, 3]]
    left, right, bottom, top = desired_extent
    dx, dy = right - left, top - bottom

    area = dx * dy

    if dx > dy * factor:
        # larger width than expected, then, height shall be increased
        dy = dx/factor
    else:
        # larger height than expected, then, width shall be increased
        dx = dy*factor
        
    xc, yc = (right + left)/2, (top + bottom)/2
    
    new_extent = [xc - dx/2, xc + dx/2, yc - dy/2, yc + dy/2]

    return new_extent, area

def get_nice_max_serie(serie, default=1, max_exp=3, min_exp=0):
    """Get the maximum value of a series rounding to the nearest multiple of 10"""
    max_serie_capped = max(default, serie.max())
    exp_max = math.floor(math.log10(max_serie_capped))
    nice_exp = max(min(exp_max, max_exp), min_exp)
    nice_max_value = round(max_serie_capped, -nice_exp)
    return nice_max_value

def plot_nice_network(n, regions_onshore, figsize=figsize, max_node_size=max_node_size, ref_area_node=ref_area_node, max_line_size=max_line_size):
    """Perform a nice plot of network 'n' with onshor regions 'regions_onshore'; optional parameters apply."""
    def_crs = ccrs.EqualEarth(n.buses.x.mean())
    regions_onshore = regions_onshore.to_crs(def_crs)

    fig, ax = plt.subplots(figsize=figsize, subplot_kw={"projection": def_crs})
    gen = n.generators.groupby(["bus", "carrier"]).p_nom.sum()
    sto = n.storage_units.groupby(["bus", "carrier"]).p_nom.sum()
    buses = pd.concat([gen, sto])

    new_extent, area = get_nice_extent(regions_onshore.total_bounds)

    # Revise size of the node according to the area of the country
    tailored_node_size = max(0.01, (area / ref_area_node) * max_node_size)

    def normalize_lines(values, max_value=max_line_size, default=1, lines=n.lines.s_nom):
        return normalize_obj(values, max_value, max(lines.max(), default))

    def normalize_buses(values, max_value=tailored_node_size, default=1, buses=buses.groupby(["bus"]).sum()):
        return normalize_obj(values, max_value, max(buses.max(), default))

    nice_max_p_nom_bus = get_nice_max_serie(buses.groupby(["bus"]).sum())
    bus_sizes = [nice_max_p_nom_bus/2, nice_max_p_nom_bus]  # in MW

    nice_max_p_nom_lines = get_nice_max_serie(n.lines.s_nom)
    line_sizes = [nice_max_p_nom_lines/2, nice_max_p_nom_lines]  # in MW

    with plt.rc_context({"patch.linewidth": 0.}):
        n.plot(
            bus_sizes=normalize_buses(buses),
            bus_alpha=0.7,
            line_widths=normalize_lines(n.lines.s_nom),
            link_widths=normalize_lines(n.links.p_nom),
            line_colors="teal",
            ax=ax,
            margin=0.2,
            color_geomap=None,
            projection=def_crs,
        )
    regions_onshore.plot(
        ax=ax,
        facecolor="whitesmoke",
        edgecolor="white",
        aspect="equal",
        transform=def_crs, #ccrs.PlateCarree(),
        linewidth=0,
    )
    ax.set_extent(new_extent, crs=def_crs)
    legend_kwargs = {"loc": "upper left", "frameon": False}
    # circles legend may requite some fine-tuning
    legend_circles_dict = {"bbox_to_anchor": (1, 0.8), "labelspacing": 2.5, **legend_kwargs}
    add_legend_circles(
        ax,
        normalize_buses(bus_sizes),
        label_objs_legend(bus_sizes),
        legend_kw=legend_circles_dict,    
    )
    add_legend_lines(
        ax,
        normalize_lines(line_sizes),
        label_objs_legend(line_sizes),
        legend_kw={"bbox_to_anchor": (1, 1.), **legend_kwargs},
    )
    add_legend_patches(
        ax,
        n.carriers.color,
        n.carriers.nice_name,
        legend_kw={"bbox_to_anchor": (1, 0), **legend_kwargs, "loc":"lower left"},
    )
    # fig.tight_layout()

    return fig


### Plot the desired network
Plot the desired network specified by the scenario_name specified above

In [ ]:
n, regions_onshore = get_scenario_network_and_regions(scenario_name)

fig = plot_nice_network(
    n,
    regions_onshore,
    figsize=figsize,
    max_node_size=max_node_size,
    max_line_size=max_line_size,
)

#### Code to execute the plot for all scenarios executed in PyPSA-Earth

This code runs over all elec networks in 'pypsa-earth/networks/*/elec.nc' and plots the corresponding network.
This code is commented by default.

In [ ]:
# for elec_path in Path("./pypsa-earth/networks/").glob("*/elec.nc"):
#     scenario_name = elec_path.parent.name

#     n, regions_onshore = get_scenario_network_and_regions(scenario_name)
    

#     fig = plot_nice_network(
#         n,
#         regions_onshore,
#         figsize=figsize,
#         max_node_size=max_node_size,
#         max_line_size=max_line_size,
#     )
#     fig.savefig(path_imgs + "/brownfield_capacities_" + scenario_name + ".png", bbox_inches="tight")
#     plt.close()